In [2]:
import numpy as np
import pandas as pd

from ComputeNode import ComputeNode

In [3]:
import xarray as xr

# MDHD Model

We'll start by recreating all the model steps here in this notebook, before re-writing them with the ComputeNode package.

**Note**: in this version, I attempted to use xarrays, which are somewhat difficult to work with. I will be instead shifting over to pandas for simplicity of implementation.

## Inputs

### Directory Information

In [4]:
# File Directories
IMPORT_FOLDER = "../Data/Inputs/"
ECON_FOLDER = "Econ/"
DEMAND_SCENARIOS_FOLDER = "Demand_Scenarios/"
VEHICLE_STOCK_FOLDER = "Vehicle_Stock_Attributes/"
VMT_ENERGY_DEMAND_FOLDER = "VMT_Energy_Demand/"
REGIONAL_MODULE_FOLDER = "Regional_Module/"
OTHER_FOLDER = "Other/" # TODO: what is that?

# New Directories
INDICES_FOLDER = "../Data/Indices/"
CROSSWALKS_FOLDER = "../Data/Crosswalks/"

### File Names

In [5]:
# Economic File Names
GSP_FILE_NAME = "Economic_Activity_Indicator_202307.xlsx"
GSP_SHEET_NAME = "Regional Mid"
FUEL_PRICE_FILE_NAME = "Fuel price forecast 2023-09-22.xlsx"
FAF_FILE_NAME = "Linear_interpolation_output_06.22.22-1623 named.xlsx"

In [6]:
# Vehicle Stock and Attributes File Names
MAINTENANCE_COST_FILE_NAME = "maintenance costs v24 2022 import.xlsx"
TRUCK_PRICE_FILE_NAME = "2023 IEPR Truck Prices - C.xlsx"
FUEL_PER_TRUCK_MI_FILE_NAME = "FFUPTM_20210603.xlsx"
LOAD_FRACTION_FILE_NAME = "t21 truck loaded fraction 20210618 RMc.xlsx"
SURVIVAL_RATE_FILE_NAME = "EMFAC_Survival_Rates_.06.04.21-1411 20221129 modtxt RMc.xlsx"
ACT_ACF_ASSIGNED_FILE_NAME = "ARB estimate ACT ACF fraction of stock 20220805 RMc.xlsx"
CLASS_BY_FLEET_SIZE_TYPE_OWNER_FILE_NAME = "MDHD_Stock_CY19_20220915 plus.xlsx"

In [7]:
# Index and Crosswalk File Names
COUNTIES_FILE_NAME = "County_names.xlsx"
ZONES_FILE_NAME = "CA_zones.xlsx"
COUNTY_FAF_CROSS_FILE_NAME = "county_to_faf.xlsx"

### Indices, Counters, and Other Final Variables

The idea here is to standardize conventions across our model so we don't run into naming errors.

In [8]:
# INDICES

# Forecast years
YEARS = range(2019, 2051)

# Possible truck model vintages
MODEL_YEARS = range(1975, 2051)

# list of counties
COUNTIES = ['Alameda', 'Alpine', 'Amador', 'Butte', 'Calaveras', 'Colusa',
       'Contra Costa', 'Del Norte', 'El Dorado', 'Fresno', 'Glenn',
       'Humboldt', 'Imperial', 'Inyo', 'Kern', 'Kings', 'Lake', 'Lassen',
       'Los Angeles', 'Madera', 'Marin', 'Mariposa', 'Mendocino',
       'Merced', 'Modoc', 'Mono', 'Monterey', 'Napa', 'Nevada', 'Orange',
       'Placer', 'Plumas', 'Riverside', 'Sacramento', 'San Benito',
       'San Bernardino', 'San Diego', 'San Francisco', 'San Joaquin',
       'San Luis Obispo', 'San Mateo', 'Santa Barbara', 'Santa Clara',
       'Santa Cruz', 'Shasta', 'Sierra', 'Siskiyou', 'Solano', 'Sonoma',
       'Stanislaus', 'Sutter', 'Tehama', 'Trinity', 'Tulare', 'Tuolumne',
       'Ventura', 'Yolo', 'Yuba']

# FAF Zones
ZONES = ['CA Los A',
 'CA rem',
 'CA Sacra',
 'CA San D',
 'CA San J',
 'CA Fresno',
 'OOS_I-10_201to500',
 'OOS_I-10_over500',
 'OOS_I-15_201to500',
 'OOS_I-15_over500',
 'OOS_I-40_201to500',
 'OOS_I-40_over500',
 'OOS_I-5_201to500',
 'OOS_I-5_over500',
 'OOS_I-8_201to500',
 'OOS_I-8_over500',
 'OOS_I-8_to200',
 'OOS_I-80_201to500',
 'OOS_I-80_over500',
 'OOS_I-80_to200']

CA_ZONES = ['CA rem', 'CA San J', 'CA Sacra', 'CA Los A', 'CA Fresno', 'CA San D']

In [56]:
# FINAL VARIABLES

# Coordinate Labels
COUNTY_LABEL = 'County'
ZONE_LABEL = 'Zone'
YEAR_LABEL = 'Year'
VINTAGE_LABEL = 'Vintage'
TRUCK_CLASS_LABEL = 'Truck_class'
FUEL_TYPE_LABEL = 'Fuel_type'

# Constant Values
TRUCK_RETIREMENT_AGE = 40

### Data Imports

### Economic

In [10]:
gsp_data = pd.read_excel(IMPORT_FOLDER + ECON_FOLDER + GSP_FILE_NAME, sheet_name=GSP_SHEET_NAME)
gsp_data.head()

,County,Zone,2019,2020,2021,2022,2023,2024,2025,2026,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
0,Alameda,CA San J,160.404767,157.339722,168.745442,166.408824,170.307781,173.731745,178.683926,184.232453,...,259.812856,265.229528,270.733966,276.347346,282.091207,287.881132,293.824507,300.183569,306.741765,313.442232
1,Alpine,CA rem,0.235178,0.207040,0.219193,0.224208,0.227723,0.227719,0.230208,0.233819,...,0.290995,0.295237,0.299536,0.303867,0.308217,0.312477,0.316835,0.321617,0.326499,0.331166
2,Amador,CA rem,2.095760,2.073996,2.099969,2.100453,2.160152,2.163390,2.199133,2.240731,...,2.770305,2.808531,2.848202,2.889100,2.930612,2.971606,3.013864,3.060628,3.108769,3.156694
3,Butte,CA rem,14.061553,13.291727,13.615938,13.218811,13.489031,13.729690,14.080265,14.481030,...,19.631352,19.989597,20.354803,20.725771,21.102527,21.479162,21.867346,22.286651,22.720523,23.161941
4,Calaveras,CA rem,1.591438,1.613576,1.668310,1.694217,1.739392,1.744850,1.769649,1.799966,...,2.135361,2.158059,2.181630,2.205694,2.229704,2.253127,2.277811,2.305962,2.335095,2.363785


In [11]:
gsp_data_xr = xr.DataArray(gsp_data.iloc[:, 2:], dims=[COUNTY_LABEL, YEAR_LABEL], coords={COUNTY_LABEL: COUNTIES, YEAR_LABEL: YEARS}) # Convert data into XArray
gsp_data_xr = gsp_data_xr.assign_coords({ZONE_LABEL: (COUNTY_LABEL, gsp_data[ZONE_LABEL].to_list())}) # add additional coordinates for FAF zone
gsp_data_xr

<xarray.DataArray (County: 58, Year: 32)>
array([[1.60404767e+02, 1.57339722e+02, 1.68745442e+02, ...,
        3.00183569e+02, 3.06741765e+02, 3.13442232e+02],
       [2.35178043e-01, 2.07039608e-01, 2.19192628e-01, ...,
        3.21617449e-01, 3.26498567e-01, 3.31166353e-01],
       [2.09575982e+00, 2.07399592e+00, 2.09996906e+00, ...,
        3.06062759e+00, 3.10876873e+00, 3.15669401e+00],
       ...,
       [6.23381008e+01, 6.24288664e+01, 6.47228313e+01, ...,
        1.06840453e+02, 1.09057153e+02, 1.11321905e+02],
       [1.92782012e+01, 1.90344216e+01, 1.95994717e+01, ...,
        2.95100774e+01, 2.99662573e+01, 3.04282224e+01],
       [2.98352863e+00, 3.13524947e+00, 3.18620649e+00, ...,
        5.67588478e+00, 5.80262439e+00, 5.93278060e+00]])
Coordinates:
  * County   (County) <U15 'Alameda' 'Alpine' 'Amador' ... 'Yolo' 'Yuba'
  * Year     (Year) int32 2019 2020 2021 2022 2023 ... 2046 2047 2048 2049 2050
    Zone     (County) <U9 'CA San J' 'CA rem' 'CA rem' ... 'CA Sacra' 'CA Sacra'

### Vehicle Stock and Attributes

In [12]:
truck_survival_rate = pd.read_excel(IMPORT_FOLDER + VEHICLE_STOCK_FOLDER + SURVIVAL_RATE_FILE_NAME)

In [13]:
truck_survival_rate.head()

,vintage,Zone,TruckClass2021,fuelType,2019,2020,2021,2022,2023,2024,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
0,1975,CA Fresno,GVWR3,Diesel-Electric Hybrid,1,0.667,0.333,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1975,CA Fresno,GVWR3,diesel,1,0.667,0.333,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1975,CA Fresno,GVWR3,gasoline,1,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1975,CA Fresno,GVWR3,hybrid,1,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1975,CA Fresno,GVWR3,propane,1,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
truck_survival_rate['vintage']

0        1975
1        1975
2        1975
3        1975
4        1975
         ... 
34425    2050
34426    2050
34427    2050
34428    2050
34429    2050
Name: vintage, Length: 34430, dtype: int64

In [49]:
truck_survival_rate = pd.read_excel(IMPORT_FOLDER + VEHICLE_STOCK_FOLDER + SURVIVAL_RATE_FILE_NAME)
truck_survival_rate = truck_survival_rate.rename({'vintage': VINTAGE_LABEL,
                            'Zone': ZONE_LABEL,
                            'TruckClass2021': TRUCK_CLASS_LABEL,
                            'fuelType': FUEL_TYPE_LABEL}, axis=1)
truck_survival_rate = truck_survival_rate.set_index([VINTAGE_LABEL, ZONE_LABEL, TRUCK_CLASS_LABEL, FUEL_TYPE_LABEL])
truck_survival_rate

2019   2020   2021  \
Vintage Zone      Truck_class Fuel_type                                    
1975    CA Fresno GVWR3       Diesel-Electric Hybrid     1  0.667  0.333   
                              diesel                     1  0.667  0.333   
                              gasoline                   1  0.000  0.000   
                              hybrid                     1  0.000  0.000   
                              propane                    1  0.000  0.000   
...                                                    ...    ...    ...   
2050    CA Los A  GVWR8 IRP   electric                   1  1.000  1.000   
        CA Sacra  GVWR8 IRP   electric                   1  1.000  1.000   
        CA San D  GVWR8 IRP   electric                   1  1.000  1.000   
        CA San J  GVWR8 IRP   electric                   1  1.000  1.000   
        CA rem    GVWR8 IRP   electric                   1  1.000  1.000   

                                                      2022  2023  2024  2025  \
Vintage Zone      Truck_class Fuel_type                                        
1975    CA Fresno GVWR3       Diesel-Electric Hybrid   0.0   0.0   0.0   0.0   
                              diesel                   0.0   0.0   0.0   0.0   
                              gasoline                 0.0   0.0   0.0   0.0   
                              hybrid                   0.0   0.0   0.0   0.0   
                              propane                  0.0   0.0   0.0   0.0   
...                                                    ...   ...   ...   ...   
2050    CA Los A  GVWR8 IRP   electric                 1.0   1.0   1.0   1.0   
        CA Sacra  GVWR8 IRP   electric                 1.0   1.0   1.0   1.0   
        CA San D  GVWR8 IRP   electric                 1.0   1.0   1.0   1.0   
        CA San J  GVWR8 IRP   electric                 1.0   1.0   1.0   1.0   
        CA rem    GVWR8 IRP   electric                 1.0   1.0   1.0   1.0   

                                                      2026  2027  2028  ...  \
Vintage Zone      Truck_class Fuel_type                                 ...   
1975    CA Fresno GVWR3       Diesel-Electric Hybrid   0.0   0.0   0.0  ...   
                              diesel                   0.0   0.0   0.0  ...   
                              gasoline                 0.0   0.0   0.0  ...   
                              hybrid                   0.0   0.0   0.0  ...   
                              propane                  0.0   0.0   0.0  ...   
...                                                    ...   ...   ...  ...   
2050    CA Los A  GVWR8 IRP   electric                 1.0   1.0   1.0  ...   
        CA Sacra  GVWR8 IRP   electric                 1.0   1.0   1.0  ...   
        CA San D  GVWR8 IRP   electric                 1.0   1.0   1.0  ...   
        CA San J  GVWR8 IRP   electric                 1.0   1.0   1.0  ...   
        CA rem    GVWR8 IRP   electric                 1.0   1.0   1.0  ...   

                                                      2041  2042  2043  2044  \
Vintage Zone      Truck_class Fuel_type                                        
1975    CA Fresno GVWR3       Diesel-Electric Hybrid   0.0   0.0   0.0   0.0   
                              diesel                   0.0   0.0   0.0   0.0   
                              gasoline                 0.0   0.0   0.0   0.0   
                              hybrid                   0.0   0.0   0.0   0.0   
                              propane                  0.0   0.0   0.0   0.0   
...                                                    ...   ...   ...   ...   
2050    CA Los A  GVWR8 IRP   electric                 1.0   1.0   1.0   1.0   
        CA Sacra  GVWR8 IRP   electric                 1.0   1.0   1.0   1.0   
        CA San D  GVWR8 IRP   electric                 1.0   1.0   1.0   1.0   
        CA San J  GVWR8 IRP   electric                 1.0   1.0   1.0   1.0   
        CA rem    GVWR8 IRP   electr

In [77]:
view = ((truck_survival_rate_xr.coords[VINTAGE_LABEL] <= truck_survival_rate_xr.coords[YEAR_LABEL]) & (truck_survival_rate_xr.coords[YEAR_LABEL] - truck_survival_rate_xr.coords[VINTAGE_LABEL] <= TRUCK_RETIREMENT_AGE)).to_pandas()
view.head()

Year,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
Vintage,,,,,,,,,,,,,,,,,,,,,
1975,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1976,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1977,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1978,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1979,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [78]:
truck_survival_rate_xr = xr.DataArray(truck_survival_rate, [('stacked', truck_survival_rate.index), (YEAR_LABEL, YEARS)])
truck_survival_rate_xr = truck_survival_rate_xr.unstack('stacked')
truck_survival_rate_xr = truck_survival_rate_xr.where(((truck_survival_rate_xr.coords[VINTAGE_LABEL] <= truck_survival_rate_xr.coords[YEAR_LABEL]) & (truck_survival_rate_xr.coords[YEAR_LABEL] - truck_survival_rate_xr.coords[VINTAGE_LABEL] <= TRUCK_RETIREMENT_AGE)),
                             truck_survival_rate_xr,
                             0)
truck_survival_rate_xr.sel({ZONE_LABEL: 'CA Los A', TRUCK_CLASS_LABEL: 'GVWR4and5', FUEL_TYPE_LABEL: 'diesel'}).to_pandas().T

Year,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
Vintage,,,,,,,,,,,,,,,,,,,,,
1975,1.0,0.667,0.333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1976,1.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1977,1.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1978,1.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1979,1.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046,1.0,1.000,1.000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2047,1.0,1.000,1.000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2048,1.0,1.000,1.000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [38]:
truck_survival_rate_xr = xr.DataArray(truck_survival_rate.iloc[:, 4:], dims=[VINTAGE_LABEL, YEAR_LABEL])
truck_survival_rate_xr['Vintage'] = truck_survival_rate['vintage'].to_list()
truck_survival_rate_xr = truck_survival_rate_xr.assign_coords({ZONE_LABEL: (VINTAGE_LABEL, truck_survival_rate['Zone']),
                                                               TRUCK_CLASS_LABEL: (VINTAGE_LABEL, truck_survival_rate['TruckClass2021']),
                                                               FUEL_TYPE_LABEL: (VINTAGE_LABEL, truck_survival_rate['fuelType'])})
truck_survival_rate_xr

<xarray.DataArray (Vintage: 34430, Year: 32)>
array([[1.   , 0.667, 0.333, ..., 0.   , 0.   , 0.   ],
       [1.   , 0.667, 0.333, ..., 0.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
       ...,
       [1.   , 1.   , 1.   , ..., 1.   , 1.   , 1.   ],
       [1.   , 1.   , 1.   , ..., 1.   , 1.   , 1.   ],
       [1.   , 1.   , 1.   , ..., 1.   , 1.   , 1.   ]])
Coordinates:
  * Vintage      (Vintage) int32 1975 1975 1975 1975 ... 2050 2050 2050 2050
  * Year         (Year) object 2019 2020 2021 2022 2023 ... 2047 2048 2049 2050
    Zone         (Vintage) object 'CA Fresno' 'CA Fresno' ... 'CA rem'
    Truck_class  (Vintage) object 'GVWR3' 'GVWR3' ... 'GVWR8 IRP' 'GVWR8 IRP'
    Fuel_type    (Vintage) object 'Diesel-Electric Hybrid' ... 'electric'

In [40]:
array = xr.DataArray(
    np.random.randn(2, 3), coords=[("x", ["a", "b"]), ("y", [0, 1, 2])]
)
array

<xarray.DataArray (x: 2, y: 3)>
array([[ 0.52009251, -1.11615487, -0.33234972],
       [-0.23921306,  0.23460294,  0.28696216]])
Coordinates:
  * x        (x) <U1 'a' 'b'
  * y        (y) int32 0 1 2

In [41]:
stacked = array.stack(z=("x", "y"))
stacked

<xarray.DataArray (z: 6)>
array([ 0.52009251, -1.11615487, -0.33234972, -0.23921306,  0.23460294,
        0.28696216])
Coordinates:
  * z        (z) object MultiIndex
  * x        (z) <U1 'a' 'a' 'a' 'b' 'b' 'b'
  * y        (z) int32 0 1 2 0 1 2

In [1]:
truck_survival_rate_xr = xr.DataArray(truck_survival_rate.iloc[:, 4:], coords={VINTAGE_LABEL: truck_survival_rate['vintage'],
                                                                             ZONE_LABEL: truck_survival_rate['Zone'],
                                                                             TRUCK_CLASS_LABEL: truck_survival_rate['TruckClass2021'],
                                                                             })
truck_survival_rate_xr

NameError: name 'xr' is not defined

## Regional Forecast

In [22]:
gsp_summed_by_zone = gsp_data_xr.groupby(ZONE_LABEL).sum() # Sum GSP by zone
gsp_summed_by_zone = gsp_summed_by_zone.sel(Zone=gsp_data_xr.Zone) # Broadcast GSP back to county level

county_percents_of_faf = gsp_data_xr / gsp_summed_by_zone
county_percents_of_faf.to_pandas().head()

Year,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
County,,,,,,,,,,,,,,,,,,,,,
Alameda,0.154719,0.152735,0.150745,0.148847,0.148550,0.148353,0.148439,0.148575,0.148811,0.149016,...,0.150344,0.150303,0.150233,0.150138,0.150031,0.149917,0.149780,0.149631,0.149471,0.149302
Alpine,0.000778,0.000693,0.000728,0.000763,0.000760,0.000748,0.000739,0.000731,0.000725,0.000720,...,0.000690,0.000689,0.000687,0.000686,0.000684,0.000683,0.000681,0.000679,0.000678,0.000675
Amador,0.006937,0.006939,0.006979,0.007143,0.007212,0.007108,0.007056,0.007003,0.006953,0.006917,...,0.006568,0.006550,0.006535,0.006521,0.006506,0.006492,0.006478,0.006466,0.006453,0.006439
Butte,0.046541,0.044473,0.045251,0.044955,0.045032,0.045109,0.045178,0.045257,0.045338,0.045420,...,0.046540,0.046620,0.046701,0.046778,0.046851,0.046926,0.047004,0.047083,0.047163,0.047244
Calaveras,0.005267,0.005399,0.005544,0.005762,0.005807,0.005733,0.005678,0.005625,0.005575,0.005533,...,0.005062,0.005033,0.005005,0.004978,0.004950,0.004922,0.004896,0.004872,0.004847,0.004821
